# CSE206 GroupF
## MemeChatbot: building an interactive chatbot using ANN


<img src="https://media4.giphy.com/media/VdoNp9BUcnXRowPaeW/giphy.gif?cid=49972e292ngfnacddfo7876rqpeju7jmwcyysvmu8snmjdwd&rid=giphy.gif&ct=" style="margin:auto"/>

## Section1: Introduction

1. Evolution

## Section2: Text-based chatbot

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


#for intent in data['intents']:
#    for pattern in intent['patterns']:
#        training_sentences.append(pattern)
#        training_labels.append(intent['tag'])
#    responses.append(intent['responses'])
    
#    if intent['tag'] not in labels:
#        labels.append(intent['tag'])

for intent in data['intents']:
    for pattern in intent['text']:
        training_sentences.append(pattern)
        training_labels.append(intent['intent'])
    responses.append(intent['responses'])
    
    if intent['intent'] not in labels:
        labels.append(intent['intent'])
        
num_classes = len(labels)

In [3]:

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 22)                374       
                                                                 
Total params: 16,918
Trainable params: 16,918
Non-trainable params: 0
____________________________________________________

2022-05-11 21:56:48.960837: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
5/5 [==============================] - 1s 2ms/step - loss: 3.0918 - accuracy: 0.0420
Epoch 2/500
5/5 [==============================] - 0s 1ms/step - loss: 3.0907 - accuracy: 0.0420
Epoch 3/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0898 - accuracy: 0.0420
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0892 - accuracy: 0.0629
Epoch 5/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0884 - accuracy: 0.0979
Epoch 6/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0878 - accuracy: 0.0979
Epoch 7/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0869 - accuracy: 0.1189
Epoch 8/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0858 - accuracy: 0.1189
Epoch 9/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0849 - accuracy: 0.1399
Epoch 10/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0836 - accuracy: 0.1399
Epoch 11/500
5/5 [===========

In [7]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
#with open('tokenizer.pickle', 'wb') as handle:
#    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
#with open('label_encoder.pickle', 'wb') as ecn_file:
#    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

2022-05-11 21:56:55.860242: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: chat_model/assets


# Section3: GIF-based chatbot

In [8]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    #with open('tokenizer.pickle', 'rb') as handle:
    #    tokenizer = pickle.load(handle)

    # load label encoder object
    #with open('label_encoder.pickle', 'rb') as enc:
    #    lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            #if i['tag'] == tag:
            if i['intent'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: ChatBot: Hi human, please tell me your GeniSys user
User: ChatBot: Any time!
User: 

# Section4: Translator (optional)

# Conclusion

# Works Cited
- Amila Viraj "How To Build Your Own Chatbot Using Deep Learning" https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281?gi=504cc6212045
- LSTM is dead. Long Live Transformers! https://youtube.com/watch?v=S27pHKBEp30